In [ ]:
%matplotlib widget
import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np
import json
import warnings
import matplotlib.pyplot as plt
from IPython import display

import main
import funcs  
warnings.filterwarnings("ignore", message="delta_grad == 0.0. Check if the approximated function is linear.")

def plot(dataset,ticker,budget,datefrom,dateto):
    dt=datefrom
    dt2=dateto
    initial_budget=budget
    path='./Data/'
    df=pd.read_parquet(path+'EWM/'+dataset)
    prices=pd.read_parquet(path+'Parquet/'+dataset)
    fechas=[x for x in list(df.index) if x>=dt and x<dt2]
    s=[ticker]
    final_df=pd.DataFrame(columns=['Markowitz','Benchmark'])
    final_df.index.name='Fecha'
    index=funcs.download_and_fill_index(ticker)
    index0=index.loc[[fechas[0]]][0]
    for f in fechas:
        #print(f,budget)
        partial_df=pd.DataFrame(columns=['Markowitz','Benchmark'],index=[f],data=[[budget/initial_budget,index.loc[[f]][0]/index0]])
        partial_df.index.name='Fecha'
        final_df=pd.concat([final_df,partial_df])
        plt.gca().cla() 
        plt.plot(final_df.index,final_df['Markowitz'],label='Markowitz')
        plt.plot(final_df.index,final_df['Benchmark'],label='Benchmark')
        plt.legend()
        display.clear_output(wait=True)
        display.display(plt.gcf()) 
        j=funcs.generateJSON(dataset,f,budget)
        sol=main.run(json.loads(j)['data'],[],[])
        budget=np.sum(np.array(prices.shift(-1).loc[[f]])[0]*sol['real_optimization']['solution'])
plot('IBEX','^IBEX',1000000,datetime(2020,1,1),datetime(2022,1,1))